# Segmenting and Clustering Neighborhoods in Toronto

##### by Luca Mancino 

In this notebook, I will explore, segment, and cluster the neighborhoods in the city of Toronto. Unlike NYC, the neighborhood data in not readily available on the web. For the purpose, I will use a Wikipedia page containing all the information needed to explore and cluster the neighborhoods in Toronto.

**INSTRUCTIONS**:
    1) run '**pip install wikipedia**' to use the Wikipedia library; 
    2) run '**pip install lxml**'. 
    
These steps are fundamental to create a Pandas dataframe starting from a Wikipedia table. Once the packages have been installed, do not compute them again. 

In [1]:
# To use the Wikipedia library, there is the need to install it. 
from pip._internal import main as _main

package_names=['wikipedia'] #packages to install
_main(['install'] + package_names + ['--upgrade']) 

Requirement already up-to-date: wikipedia in /home/jupyterlab/conda/envs/python/lib/python3.6/site-packages (1.4.0)


0

In [2]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install shapely

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Start by creating a new Notebook for this assignment.
# setup import
import pandas as pd 
import wikipedia as wp
from bs4 import BeautifulSoup

In [6]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html, header = 0)[0]

# The next line code is used to drop the rows containing 'Not assigned' values in the 'Borough' column
df_1 = df[df.Borough != 'Not assigned']

In [7]:
# Shape of the 'df' dataset (which is not 'cleaned')
df.shape

(180, 3)

In [8]:
# Shape of the 'df_1' dataset obtained by dropping out the rows containing 'Not assigned' values in the 'Borough' colums
df_1.shape

(103, 3)

Note that more than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma. I will call df_2 the new dataset.

In [9]:
df_2=df_1.groupby(['Postal code', 'Borough'])['Neighborhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

In [10]:
df_2.iterrows()

<generator object DataFrame.iterrows at 0x7fedf4e95518>

In [11]:
for index, row in df_2.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

In [12]:
df_2

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...


In [13]:
# Shape of the 'df_2' dataset 
df_2.shape

(103, 3)

In what follows, we use some geospatial data (http://cocl.us/Geospatial_data) to merge the geographical coordinates of the neighborhoods in the 'df_2' dataset. For the purpose, we import 'requests' and 'io' as shown here:

In [14]:
import requests
import io

In [15]:
geo_url="http://cocl.us/Geospatial_data" # url address
requests_geo_url=requests.get(geo_url).content # request

In [16]:
# df_PC_L_L is a dataset with 'Postal Code', 'Latitude', and 'Longitude' as columns
df_PC_L_L=pd.read_csv(io.StringIO(requests_geo_url.decode('utf-8')))
# We can use the .head() method to print the first five rows of the dataset
df_PC_L_L.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
df_2.columns=['PostalCode', 'Borough', 'Neighborhood'] # From 'Postal Code' to 'PostalCode' as required
df_PC_L_L.columns=['PostalCode', 'Latitude', 'Longitude'] # From 'Postal Code' to 'PostalCode' as required

In [18]:
# We use Pandas to merge the two datasets using 'PostalCode' as reference:
# the two dataset are merged associating the correct 'Latitude' and 'Longitude' to the PostalCode
df_3 = pd.merge(df_2, df_PC_L_L, on='PostalCode')

In [19]:
# We use the .head() method to visualise the first five rows of the df_3 dataset
df_3.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [20]:
from shapely.geometry import Point # Remember to compute 'pip install shapely'
import geopandas as gpd # Remember to compute 'pip install geopandas'
import matplotlib.pyplot as plt

generated new fontManager


In [28]:
# We create a list containing the Longitude and Latitude coordinates from the df_3 dataset
df_3_coordinates=list(zip(df_3.Longitude,df_3.Latitude))

In [29]:
# We create a GeoPandas dataframe
gdf = gpd.GeoDataFrame(df_3_coordinates)

In [30]:
cities = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))

# cities is a dataset containing the 'name' of the city and the 'geometry'
cities.head()

,name,geometry
0,Vatican City,POINT (12.45339 41.90328)
1,San Marino,POINT (12.44177 43.93610)
2,Vaduz,POINT (9.51667 47.13372)
3,Luxembourg,POINT (6.13000 49.61166)
4,Palikir,POINT (158.14997 6.91664)


In [38]:
i=cities[cities['name']=='Toronto']
print(i)

Empty GeoDataFrame
Columns: [name, geometry]
Index: []


In [31]:
cities.shape

(202, 2)

In [32]:
# First row of the 'cities' dataset
cities[cities.name == "Vatican City"]

,name,geometry
0,Vatican City,POINT (12.45339 41.90328)


In [33]:
# First row of the 'cities' dataset
cities[cities.name == "Toronto"]

,name,geometry
